In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import csv
import matplotlib.pyplot as plt
import numpy as np
from itertools import cycle
from scipy.spatial import distance

dir_base = '/content/drive/My Drive/SBRC 2024/results_sbrc_models_fl-cfa/'

Mounted at /content/drive


In [2]:
real_locations = []
real_orientations = []
predicted_gru_locations = []
predicted_gru_orientations = []
predicted_esn_locations = []
predicted_esn_orientations = []
predicted_lstm_locations = []
predicted_lstm_orientations = []

In [3]:
def read_csv_plot_files(user_id, model):
  file_name = f"{user_id}_{model}_model_tracking_data.csv"
  file_path = os.path.join(dir_base, file_name)

  user_real_locations         = []
  user_real_orientations      = []
  user_predicted_locations    = []
  user_predicted_orientations = []

  with open(file_path, 'r') as file:
      reader = csv.reader(file)

      for row in reader:
          real_data = np.array([float(coord) for coord in row[1].strip('[]').split()])
          user_real_locations.append([real_data[0], real_data[1]])
          user_real_orientations.append(real_data[2])

          predicted_data = np.array([float(coord) for coord in row[2].strip('[]').split()])
          user_predicted_locations.append([predicted_data[0], predicted_data[1]])
          user_predicted_orientations.append(predicted_data[2])

  return user_real_locations, user_real_orientations, user_predicted_locations, user_predicted_orientations

In [4]:
for user_id in range(0,10):
  user_real_locations, user_real_orientations, user_predicted_locations, user_predicted_orientations = \
    read_csv_plot_files(user_id, 'gru')

  real_locations.append(user_real_locations)
  real_orientations.append(user_real_orientations)
  predicted_gru_locations.append(user_predicted_locations)
  predicted_gru_orientations.append(user_predicted_orientations)

for user_id in range(0,10):
  _, _, user_predicted_locations, user_predicted_orientations = \
    read_csv_plot_files(user_id, 'esn')

  predicted_esn_locations.append(user_predicted_locations)
  predicted_esn_orientations.append(user_predicted_orientations)

for user_id in range(0,10):
  _, _, user_predicted_locations, user_predicted_orientations = \
    read_csv_plot_files(user_id, 'lstm')

  predicted_lstm_locations.append(user_predicted_locations)
  predicted_lstm_orientations.append(user_predicted_orientations)

In [ ]:
usuarios = np.arange(1, 11)

orientacao_gru = []
orientacao_lstm = []
orientacao_esn = []

for user_id in range(0,10):
    orientacao_gru.append([abs(pred - real) for pred, real in zip(predicted_gru_orientations[user_id], real_orientations[user_id])])
    orientacao_lstm.append([abs(pred - real) for pred, real in zip(predicted_lstm_orientations[user_id], real_orientations[user_id])])
    orientacao_esn.append([abs(pred - real) for pred, real in zip(predicted_esn_orientations[user_id], real_orientations[user_id])])

orientacao_gru = np.array(orientacao_gru)
orientacao_lstm = np.array(orientacao_lstm)
orientacao_esn = np.array(orientacao_esn)

# Calcular a média da orientação para cada usuário e cada rede
media_gru = (orientacao_gru / 225).tolist()[0]
media_lstm = (orientacao_lstm / 326).tolist()[0]
media_esn = (orientacao_esn / 269).tolist()[0]

# Plotar gráfico de barras agrupadas
barWidth = 0.25
r1 = np.arange(len(usuarios))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]

fig1 = plt.figure(figsize=(10, 5))

plt.rcParams['font.size'] = '20'
plt.tight_layout()

plt.bar(r3, media_esn, color='#33adff', width=barWidth, edgecolor='grey', label='ESN')
plt.bar(r1, media_gru, color='#e6b800', width=barWidth, edgecolor='grey', label='GRU')
plt.bar(r2, media_lstm, color='#cc0066', width=barWidth, edgecolor='grey', label='LSTM')

# Ajustar layout
plt.xlabel('Usuários', fontsize=24)
plt.ylabel('Exatidão/Convergência', fontsize=24)
plt.xticks([r + barWidth for r in range(len(usuarios))], usuarios)
plt.legend()

plt.savefig(dir_base + "Orientação - Comparativo entre RNNs.pdf", bbox_inches='tight')

In [ ]:
usuarios = np.arange(1, 11)

localizacao_gru = []
localizacao_lstm = []
localizacao_esn = []

for user_id in range(0,10):
    localizacao_gru.append([distance.euclidean((pred[0], pred[1]),(real[0],real[1])) for pred, real in zip(predicted_gru_locations[user_id], real_locations[user_id])])
    localizacao_lstm.append([distance.euclidean((pred[0], pred[1]),(real[0],real[1])) for pred, real in zip(predicted_lstm_locations[user_id], real_locations[user_id])])
    localizacao_esn.append([distance.euclidean((pred[0], pred[1]),(real[0],real[1])) for pred, real in zip(predicted_esn_locations[user_id], real_locations[user_id])])

localizacao_gru = np.array(localizacao_gru)
localizacao_lstm = np.array(localizacao_lstm)
localizacao_esn = np.array(localizacao_esn)

# Calcular a média da orientação para cada usuário e cada rede
media_gru = (localizacao_gru / 225).tolist()[0]
media_lstm = (localizacao_lstm / 326).tolist()[0]
media_esn = (localizacao_esn / 269).tolist()[0]

# Plotar gráfico de barras agrupadas
barWidth = 0.25
r1 = np.arange(len(usuarios))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]

fig2 = plt.figure(figsize=(10, 5))

plt.rcParams['font.size'] = '20'
plt.tight_layout()

plt.bar(r3, media_esn, color='#33adff', width=barWidth, edgecolor='grey', label='ESN')
plt.bar(r1, media_gru, color='#e6b800', width=barWidth, edgecolor='grey', label='GRU')
plt.bar(r2, media_lstm, color='#cc0066', width=barWidth, edgecolor='grey', label='LSTM')

# Ajustar layout
plt.xlabel('Usuários', fontsize=24)
plt.ylabel('Exatidão/Convergência', fontsize=24)
plt.xticks([r + barWidth for r in range(len(usuarios))], usuarios)
#plt.title('Comparação do Erro de Localização(Precisão) por Usuário/Rodadas para Convergência - GRU, LSTM, ESN')
plt.legend()

#plt.show()

plt.savefig(dir_base + "Localização - Comparativo entre RNNs.pdf", bbox_inches='tight')